In [13]:
import numpy as np

def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName,'r')
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine))   #map将里面的值变为float 
        dataMat.append(fltLine)
    return dataMat

def distEclud(vecA,vecB):
    return np.sqrt(np.sum(np.power(vecA-vecB,2)))

#最开始 随机选 k 个 中心点
def randCent(dataSet,k):
    n = np.shape(dataSet)[1]
    centroids = np.mat(np.zeros((k,n)))
    for j in range(n):
        minJ = np.min(dataSet[:,j])
        rangeJ = float(np.max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ +rangeJ * np.random.rand(k,1)  #随机一个 k行 1 列的矩阵
    return centroids     

def kMeans(dataSet,k,distMeans = distEclud,createCent=randCent):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroids = createCent(dataSet,k)
    clusterChanged  = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = np.inf
            minIndex = -1
            for j in range(k):
                distJI = distMeans(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex:    # 跟新质心后，在进行while循环，如果最小值都相同，则推出，如果最小值存在不同，则需要再循环
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)
        for cent in range(k):
            #np.nonzero 返回不为零的下标
            ptsInClust = dataSet[np.nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = np.mean(ptsInClust,axis = 0)
    return centroids, clusterAssment

dataMat = np.mat(loadDataSet('E:/pythonNotebook/Unsupervised_learning/testSet.txt'))
myCentroids,clustAssing = kMeans(dataMat,4)

[[ 0.68461144 -3.26775369]
 [ 0.41329784 -2.00846352]
 [ 0.44501165  1.0128835 ]
 [-0.8375292   4.92363415]]
[[ 1.278582   -3.45559463]
 [-1.82930245 -2.37206195]
 [ 1.433159    2.27756348]
 [-1.30462025  3.48064   ]]
[[ 2.7481024  -2.90572575]
 [-3.53973889 -2.89384326]
 [ 2.53499033  2.93781533]
 [-2.46154315  2.78737555]]
[[ 2.65077367 -2.79019029]
 [-3.53973889 -2.89384326]
 [ 2.6265299   3.10868015]
 [-2.46154315  2.78737555]]


In [15]:
'''
    二分 K-均值 聚类算法
    1、首先将所有点作为一个簇，然后将该簇一分为二，之后选择其中一个簇继续进行划分，选择哪一个簇进行取决于其划分是否可以最大程度降低SSE的值
    2、选择SSE最大的簇进行划分
'''
def biKmeans(dataSet,k,disMeans = distEclud):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataSet,axis = 0).tolist()[0]
    centList = [cemtroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeans(np.mat(centroid0),dataSet[j,:])**2
    while(len(centList) < k ):
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[np.nonzero(clusterAssment[:,0].A ==i)[0],:]
            centroidMat,splitClustAss = kMeans(ptsInCurrCluster,2,distMeans)
            sseSplit = np.sum(splitClustAss[:,1])                     #进行划分后的子集的总SSE
            sseNotSplit = np.sum(clusterAssment[np.nonzero(clusterAssment[:,0].A!=i)[0],1]) #未进行划分的子集的总SSE
            
            print('sseSplit , and notSplit',sseSplit,sseNotSplit)
            
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[np.nonzero(bestClustAss[:,0].A==1)[0],0] = len(centList)   #将划分后子集的一部分 序号标记为新的序号（+1）
        bestClustAss[no.nonzero(bestClustAss[:,0].A==0)[0],0] = bestCentToSplit #将划分后子集的另一部分 更新原来的序号
        
        print('the bestCentToSplit is :', bestCentToSplit)
        print('the len of bestClustAss is :', len(bestClustAss))
        
        centList[bestCentToSplit] = bestNewCents[0,:]        #跟新质点的序号
        centList.append(bestNewCents[1,:])                   #对另一个质点 新加序号
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
        
        return np.mat(centList),clusterAssment